# Topic: Predicting PM2.5 Value in Next Hour
Description: Using time series concept and the Linear Regression, Random Forest Regression model to predict the  HsinChu City PM 2.5 value in next hour by past 6 hours data.
Please refer to the Datasets: 107年新竹站_20190315.xls

## Step 1: Import the needed tools and datasets
Firstly, import all tools and dataset which are needed, including the the tools for manage dataframe, array, the model constructing tools from scikit-learn, and the MAE for showing the accuacy rate. As the dataset shows, there are 27 columns in the dataframe. Including "Date", "Site Location", "18 Testing Features" and "Values of 24 Hours." In this case, the Characteristic Variable, also called Variable X should be past 6 hours value of each "17 Testing Item (exclude PM2.5)", and the Target Variable is the 7th hour value of PM2.5 Item. 
___
Since the first data of 10/1 is not divisible by 18 when managing time series data in the next step, we found that there are only 15 testing features for one day in July in the original data set, so we decided to take out the data of October-December first to create an new dataframe, named "temp."

In [1]:
#import library
import pandas as pd
import numpy as np
from pandas.core.frame import DataFrame
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [2]:
#load data
data = pd.read_excel('107年新竹站_20190315.xls')
data

,日期,測站,測項,00,01,02,03,04,05,06,...,14,15,16,17,18,19,20,21,22,23
0,2018/01/01,新竹,AMB_TEMP,15.4,15.4,15.3,15.3,15.3,15.3,15.3,...,21.8,21,19.9,18.6,17.9,17.9,17.8,17.6,17.6,17.7
1,2018/01/01,新竹,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.9,1.9,1.8,1.9
2,2018/01/01,新竹,CO,0.34,0.32,0.34,0.32,0.29,0.29,0.32,...,0.43,0.44,0.46,0.55,0.57,0.59,0.6,0.58,0.52,0.49
3,2018/01/01,新竹,NMHC,0.06,0.06,0.06,0.06,0.06,0.05,0.06,...,0.09,0.09,0.1,0.13,0.12,0.12,0.13,0.12,0.1,0.09
4,2018/01/01,新竹,NO,0.2,0,-0.3,-0.3,-0.1,0.2,0,...,1.1,1.2,0.7,0.4,0.3,0.2,0.2,0.2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6559,2018/12/31,新竹,THC,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.9,1.9,1.9,1.9,1.9,1.9,1.9,1.9,1.9,1.8
6560,2018/12/31,新竹,WD_HR,57,57,56,57,50,57,67,...,52,53,56,62,68,73,69,68,66,71
6561,2018/12/31,新竹,WIND_DIREC,56,60,54,60,49,66,70,...,53,51,61,71,61,73,68,53,64,79
6562,2018/12/31,新竹,WIND_SPEED,4,4.8,3.3,3.3,4.7,3.7,3.8,...,5.8,4.4,5.3,4.9,4.4,5,4.9,4.7,5,4.8


In [3]:
#extract Oct, Nov, Dec
date = data['日期'] >= "2018/10/01"
daten = pd.DataFrame(data[date])
daten.to_csv('temperature.csv', index=False, encoding = 'utf-8')
temp = pd.read_csv('temperature.csv')
temp

,日期,測站,測項,00,01,02,03,04,05,06,...,14,15,16,17,18,19,20,21,22,23
0,2018/10/01,新竹,AMB_TEMP,24.7,24.3,23.9,22.8,22.4,22.3,22.7,...,30.9,30.1,29,27.7,26.8,26,25.7,25.6,25.4,25
1,2018/10/01,新竹,CH4,1.8,1.8,1.8,1.9,1.9,1.8,1.8,...,1.9,1.9,1.9,1.9,1.9,1.9,1.9,1.9,1.9,1.8
2,2018/10/01,新竹,CO,0.26,0.16,0.14,0.13,0.14,0.15,0.18,...,0.29,0.3,0.34,0.41,0.48,0.46,0.38,0.36,0.3,0.24
3,2018/10/01,新竹,NMHC,0.05,0.05,0.05,0.05,0.05,0.06,0.06,...,0.1,0.11,0.14,0.16,0.16,0.15,0.1,0.09,0.05,0.04
4,2018/10/01,新竹,NO,0.5,0.5,0.7,0.4,0.6,0.6,0.9,...,0.7,0.8,2.2,4.5,0.6,0.5,-0.6*,0.6,1.1,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,2018/12/31,新竹,THC,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.9,1.9,1.9,1.9,1.9,1.9,1.9,1.9,1.9,1.8
1652,2018/12/31,新竹,WD_HR,57,57,56,57,50,57,67,...,52,53,56,62,68,73,69,68,66,71
1653,2018/12/31,新竹,WIND_DIREC,56,60,54,60,49,66,70,...,53,51,61,71,61,73,68,53,64,79
1654,2018/12/31,新竹,WIND_SPEED,4,4.8,3.3,3.3,4.7,3.7,3.8,...,5.8,4.4,5.3,4.9,4.4,5,4.9,4.7,5,4.8


## Step 2: Data Pre-processing
### Data Correction  and Organization
Firstly, checking the row index of PM2.5. Because we will extract all PM2.5 in the next step, it's needed to check if all PM2.5 value is in the ninth row of each day. Then, creating two empty list for train and test data. Then use the functions of "For Loop" and "Append" to put each testing feature into different lists according to the time order. The result is: there will be 18 lists in the list of train with 61(days)* 24(hr) data, and 18 lists in the list of test with 61(days)* 24(hr) data. The lists are converted into Dataframe format for fulllfilling the missingl value.

In [4]:
#check row index of PM2.5
(temp[temp.values == "PM2.5"].index)%18

Int64Index([9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
            9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
            9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
            9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
            9, 9, 9, 9],
           dtype='int64')

In [5]:
#make a list for 18 items and seperate train and test
temp_train = []
temp_test = []

for i in range(18):
    temp_train.append([])
    temp_test.append([])
    
for i in range(len(temp)):
    if temp.iloc[i,0] >= "2018/10/01" and temp.iloc[i,0] <= "2018/11/30":
        for j in range(3, 27):
            temp_train[i%18].append((temp.iloc[i,j]))
    elif temp.iloc[i,0] >= "2018/12/01":
        for j in range(3, 27):
            temp_test[i%18].append((temp.iloc[i,j]))
            
#transfer to dataframe
temp_train = DataFrame(temp_train)
temp_test = DataFrame(temp_test)

### Data Cleaning
Finishing above actions, we can fullfill the missing value and Abnotmal Value. First of all, using the function "Replace" to change "NR" to "0" in rainfall. When handling the abnormal values of other features, we have to operate "Train" and "Test" data separately. Use the function "for loop" to check each column in a row with the function "rfind" in order to see if there are any outliers, and if there are, add the average of the front and back cells. After one row is finished, go to the next row and repeat the above action.

In [6]:
#fill in rainfall missing value with 0
temp_train = temp_train.replace("NR", 0)
temp_test = temp_test.replace("NR",0)

In [7]:
#fill in missing value with mean, train_data
for a in range(0, len(temp_train)):
    for i in range (1, len(temp_train.columns)):
        j = i-1
        k = i+1
        if str(temp_train.iloc[a, i]).rfind("#") !=-1 or str(temp_train.iloc[a,i]).rfind("*") !=-1 or str(temp_train.iloc[a,i]).rfind("x")!=-1 or str(temp_train.iloc[a,i]).rfind("A")!=-1 or pd.isnull(temp_train.iloc[a,i]) == True:
            while str(temp_train.iloc[a,j]).rfind("#") != -1 or str(temp_train.iloc[a,j]).rfind("*") !=-1 or str(temp_train.iloc[a,j]).rfind("x")!=-1 or str(temp_train.iloc[a,j]).rfind("A")!= -1 or pd.isnull(temp_train.iloc[a,j]) == True:
                j = j-1
            while str(temp_train.iloc[a,k]).rfind("#") != -1 or str(temp_train.iloc[a,k]).rfind("*") !=-1 or str(temp_train.iloc[a,k]).rfind("x")!=-1 or str(temp_train.iloc[a,k]).rfind("A")!= -1 or pd.isnull(temp_train.iloc[a,k]) == True:
                k = k+1
            temp_train.iloc[a, i] = str((float(temp_train.iloc[a,j]) + float(temp_train.iloc[a,k])) / 2)
            #print(temp_train.iloc[a, i], temp_train.iloc[a, k], temp_train.iloc[a, j])


In [8]:
#fill in missing value with mean, test_data
for a in range(0, len(temp_test)):
    for i in range (1, len(temp_test.columns)):
        j = i-1
        k = i+1
        if str(temp_test.iloc[a, i]).rfind("#") !=-1 or str(temp_test.iloc[a,i]).rfind("*") !=-1 or str(temp_test.iloc[a,i]).rfind("x")!=-1 or str(temp_test.iloc[a,i]).rfind("A")!=-1 or pd.isnull(temp_test.iloc[a,i]) == True:
            while str(temp_test.iloc[a,j]).rfind("#") != -1 or str(temp_test.iloc[a,j]).rfind("*") !=-1 or str(temp_test.iloc[a,j]).rfind("x")!=-1 or str(temp_test.iloc[a,j]).rfind("A")!= -1 or pd.isnull(temp_test.iloc[a,j]) == True:
                j = j-1
            while str(temp_test.iloc[a,k]).rfind("#") != -1 or str(temp_test.iloc[a,k]).rfind("*") !=-1 or str(temp_test.iloc[a,k]).rfind("x")!=-1 or str(temp_test.iloc[a,k]).rfind("A")!= -1 or pd.isnull(temp_test.iloc[a,k]) == True:
                k = k+1
            temp_test.iloc[a, i] = str((float(temp_test.iloc[a,j]) + float(temp_test.iloc[a,k])) / 2)
            #print(temp_test.iloc[a, i], temp_test.iloc[a, k], temp_test.iloc[a, j])

### Data X, Y Setting and Organization
Setting x and y seperately. For X_data, take the previous dataframe "temp_train" and drop "PM2.5" in ninth row. Firstly, create an empty list "61(days)* 24(hours) - 6" and then use three loops. In this "61* 24-6" list, put the 6 hours data in order by the 17 features. For y_data, use function "iloc" to separate the PM2.5 data in ninth row. By using functions "list", "for" and "append", the values after line 6 are stored in the list of data_y in order.

In [9]:
#split train_x
temp_train_x = temp_train.drop(index = 9)
period = 6
length = len(temp_train_x.columns) - period
train_x = []
for i in range(length):
    train_x.append([])
for i in range(length):    
    for j in range(len(temp_train_x.index)):
        for k in range(period):
            train_x[i].append(temp_train_x.iloc[j, i+k])

In [10]:
#split train_y
temp_train_y = temp_train.iloc[9,:]
train_y = []
for i in range(6, len(temp_train_y)):
    train_y.append(temp_train_y[i])

In [11]:
#split test_x
temp_test_x = temp_test.drop(index = 9)
period = 6
length = len(temp_test_x.columns) - period
test_x = []
for i in range(length):
    test_x.append([])
for i in range(length):    
    for j in range(len(temp_test_x.index)):
        for k in range(period):
            test_x[i].append(temp_test_x.iloc[j, i+k])

In [12]:
#split test_y
temp_test_y = temp_test.iloc[9,:]
test_y = []
for i in range(6, len(temp_test_y)):
    test_y.append(temp_test_y[i])

In [13]:
#transfer str to int
train_x = np.float64(train_x)
train_y = np.float64(train_y)
test_x = np.float64(test_x)
test_y = np.float64(test_y)

## Step 3: Construct Predicting Model
After finishing data pre-processing, the model can be constructed. The applied model is Linear Regression Model and Random Forest Regression Model. The fuction Regression Model is imported from Scikit-learn.

In [20]:
#construct and train linear model
lr_model = LinearRegression()
lr_model.fit(train_x, train_y)
pred_y1 = lr_model.predict(test_x)

In [21]:
#train random forest model
rf_model = RandomForestRegressor(max_depth=5, random_state=0)
rf_model.fit(train_x, train_y)
pred_y2 = rf_model.predict(test_x)

## Step 4: Reviewing Model Accuracy
In this case, MAE was used to review the model accuracy

In [24]:
#caculate error
mean_absolute_error(test_y, pred_y1)

3.8935672441414018

In [25]:
mean_absolute_error(test_y, pred_y2)

4.046533551881666